Initialize Gen and the function libraries

In [1]:
using Gen

In [19]:
cd("/Users/lilis/Documents/GitHub/HSOManalysisPackages/JuliaLanguage_Hierarchical_Models")
include(join([pwd(), "/function_library/file_modules.jl"]))
refresh_tools("function_library")
refresh_tools("gen_stepvramp_library")

Refreshed (2021-03-26_1800) GLMNet_tools.jl auto_pull_matlabdatasets.jl cluster_modules.jl feedback_modules.jl file_modules.jl hazard_modules.jl v 11:37
math_modules.jl modeling_tools.jl plotting_tools.jl probabilistic_modules.jl Refreshed (2021-03-26_1800) gen_math_modules.jl gen_plotting_modules.jl gen_stepvramp.jl runner_modules.jl svr_filemods.jl 

Enter the CSV folder for all our sessions

In [9]:
# cd("N:\\Assad Lab\\allison\\Collated Analyses\\JULIA_CSVs\\All_SNc_nosplitsesh")
cd("/Volumes/Neurobio/Assad Lab/allison/Collated Analyses/JULIA_CSVs/All_SNc_nosplitsesh")

In [22]:
B1_idx = 1:5
B2_idx = 6:13
B3_idx = 14:24
B5_idx = 75:80
B6_idx = 25:30
H3_idx = 44:51
H4_idx = 52:66
H5_idx = 67:74
H6_idx = 81:92
H7_idx = 93:98
H14_idx = 31:38
H15_idx = 39:43

39:43

In [23]:
runID = "2021_3_26_Mac"
pathIDx = [75]
# mac
collatedPath = "/Volumes/Neurobio/Assad Lab/allison/Collated Analyses/JULIA_CSVs/All_SNc_nosplitsesh"
compositesavepath="/Volumes/Neurobio/Assad Lab/allison/Collated Analyses/HIERARCHY results/3-26-21 hierarchy1"
# windows
# collatedPath = "N:\\Assad Lab\\allison\\Collated Analyses\\JULIA_CSVs\\All_SNc_nosplitsesh"
# compositesavepath="N:\\Assad Lab\\allison\\Collated Analyses\\HIERARCHY RESULTS\\3-26-21 hierarchy1"
modelpackagefunction = hierarchy_v1#SHUFFLE_DAHxmodel_logit_200hx_pkg
postprocessingfunction = template_postprocessingfunction#bootlogit_timeslice_postprocessingfunction1

(fails, results,postprocessing) = run_collated_model(collatedPath, 
    modelpackagefunction; pathIDx = pathIDx,runFails=false, failDirs=[], postprocessingfunction=postprocessingfunction,
    compositesavepath=compositesavepath, runID=runID, suppressFigures=true); 

-----------------------------------------------
	
Initializing run_collated_model for hierarchy_v1_2021_3_26_Mac
	
Found 98 sessions in /Volumes/Neurobio/Assad Lab/allison/Collated Analyses/JULIA_CSVs/All_SNc_nosplitsesh:
1. B1_SNc_13
2. B1_SNc_15
3. B1_SNc_18
4. B1_SNc_19
5. B1_SNc_20
6. B2_SNc_11
7. B2_SNc_15
8. B2_SNc_17
9. B2_SNc_18
10. B2_SNc_19
11. B2_SNc_20
12. B2_SNc_21
13. B2_SNc_9
14. B3_SNc_10
15. B3_SNc_12
16. B3_SNc_14
17. B3_SNc_15
18. B3_SNc_16
19. B3_SNc_17
20. B3_SNc_18
21. B3_SNc_19
22. B3_SNc_20
23. B3_SNc_21
24. B3_SNc_8
25. B6_SNc_10
26. B6_SNc_11
27. B6_SNc_12
28. B6_SNc_13
29. B6_SNc_7
30. B6_SNc_9
31. H14_SNc_10
32. H14_SNc_12
33. H14_SNc_14
34. H14_SNc_16
35. H14_SNc_18
36. H14_SNc_5
37. H14_SNc_7
38. H14_SNc_8
39. H15_SNc_16
40. H15_SNc_18
41. H15_SNc_5
42. H15_SNc_7
43. H15_SNc_8
44. H3_SNc_13
45. H3_SNc_15
46. H3_SNc_17
47. H3_SNc_18
48. H3_SNc_19
49. H3_SNc_20
50. H3_SNc_7
51. H3_SNc_9
52. H4_SNc_11
53. H4_SNc_13
54. H4_SNc_15
55. H4_SNc_17
56. H4_SNc_18
57

LoadError: InterruptException:

In [24]:
@gen function my_model(xs::Vector{Float64})
    slope = @trace(normal(0, 2), :slope)
    intercept = @trace(normal(0, 10), :intercept)
    for (i, x) in enumerate(xs)
        @trace(normal(slope * x + intercept, 1), "y-$i")
    end
end
function my_inference_program(xs::Vector{Float64}, ys::Vector{Float64}, num_iters::Int)
    # Create a set of constraints fixing the 
    # y coordinates to the observed y values
    constraints = choicemap()
    for (i, y) in enumerate(ys)
        constraints["y-$i"] = y
    end
    
    # Run the model, constrained by `constraints`,
    # to get an initial execution trace
    (trace, _) = generate(my_model, (xs,), constraints)
    
    # Iteratively update the slope then the intercept,
    # using Gen's metropolis_hastings operator.
    for iter=1:num_iters
        (trace, _) = metropolis_hastings(trace, select(:slope))
        (trace, _) = metropolis_hastings(trace, select(:intercept))
    end
    
    # From the final trace, read out the slope and
    # the intercept.
    choices = get_choices(trace)
    return (choices[:slope], choices[:intercept])
end
xs = [1., 2., 3., 4., 5., 6., 7., 8., 9., 10.]
ys = [8.23, 5.87, 3.99, 2.59, 0.23, -0.66, -3.53, -6.91, -7.24, -9.90]
(slope, intercept) = my_inference_program(xs, ys, 1000)
println("slope: $slope, intercept: $intercept")

LoadError: UndefVarError: select not defined